# Content Creation at Scale

## Initial Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')



import os
from dotenv import load_dotenv
import yaml
from crewai import Agent, Task, Crew, LLM

In [ ]:
load_dotenv()


## Creating Structured Output

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

we want to make sure the data is consistent when we passed them around to different agents and systems

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [ ]:
websearchtool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="azure_openai", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gpt-4o-mini",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="azure_openai", # or openai, ollama, ...
            config=dict(
                model="text-embedding-ada-002",
                deployment_name= "text-embedding-ada-002",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


## Creating Crew, Agents, and Tasks

In [ ]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=LLM(model=f'azure/gpt-4o'),
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), websearchtool],
    llm=LLM(model=f'azure/gpt-4o'),
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), websearchtool], # had to customize WebsiteSearchTool
    llm=LLM(model=f'azure/gpt-4o')
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
    llm=LLM(model=f'azure/gpt-4o')
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [ ]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

## Social Content

In [ ]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

## Blog Post

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

In [ ]:
import pandas as pd

costs = 2.50 * (content_creation_crew.usage_metrics.prompt_tokens + content_creation_crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([content_creation_crew.usage_metrics.dict()])
df_usage_metrics